In [1]:
!pip install mordred
!pip install umap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
karateclub 1.3.3 requires networkx<2.7, but you have networkx 2.8.8 which is incompatible.
karateclub 1.3.3 requires numpy<1.23.0, but you have numpy 1.25.1 which is incompatible.
karateclub 1.3.3 requires pandas<=1.3.5, but you have pandas 1.5.3 which is incompatible.


In [2]:
import os,re,sys,pickle,datetime,time,random,itertools
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from scipy import stats
import math
from tqdm import tqdm
import seaborn as sns
import umap
from multiprocessing import freeze_support
import mordred
from mordred import Calculator, descriptors
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA,NMF
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures
import rdkit
from rdkit import Chem
from rdkit.Chem import DataStructs, AllChem, MolFromSmiles, PandasTools, Descriptors, Draw, PropertyMol, rdmolfiles, rdFMCS
from rdkit import RDConfig
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Draw import MolsToGridImage, IPythonConsole, rdMolDraw2D
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions, Font
DrawingOptions.bondLineWidth=1.8
DrawingOptions.includeAtomNumbers=False
size = (150, 150)
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image
randomstate = 42

In [3]:
filepath1 = "/home/ritesh/Desktop/MOLECULES/morgan/or_agonist_update_2.csv"
mols = pd.read_csv(filepath1)
mols

,SMILES
0,CCCCOC(=O)C(C)OC(=O)CCC
1,CCCCC(=O)O
2,CC(C)CCC(=O)O
3,C1C2CC(C1C=C2)C(=O)O
4,CCCC(C)C(=O)O
...,...
66,C1=CC=C(C=C1)CC=O
67,CC\C=C/CC=O
68,CCCC(=O)OCC
69,CCC1=NC=C(N=C1C)C


In [4]:
mols = mols.drop_duplicates(subset=['SMILES'])
mols

,SMILES
0,CCCCOC(=O)C(C)OC(=O)CCC
1,CCCCC(=O)O
2,CC(C)CCC(=O)O
3,C1C2CC(C1C=C2)C(=O)O
4,CCCC(C)C(=O)O
...,...
66,C1=CC=C(C=C1)CC=O
67,CC\C=C/CC=O
68,CCCC(=O)OCC
69,CCC1=NC=C(N=C1C)C


In [5]:
mols['rdmol'] = mols['SMILES'].map(lambda x: Chem.MolFromSmiles(x))
mols['inchi'] = mols['rdmol'].map(Chem.MolToInchi)
mols = mols.drop_duplicates(subset="inchi")

print('Number of SMILES after duplicates removed:',len(mols))

mols

Number of SMILES after duplicates removed: 67


[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Proton(s) added/removed

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Charges were rearranged

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Omitted undefined stereo

[19:38:07] WARNING: Proton(s) added/removed

[19:38:07] WARNING: Omitted undefined stereo



,SMILES,rdmol,inchi
0,CCCCOC(=O)C(C)OC(=O)CCC,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ac200>,InChI=1S/C11H20O4/c1-4-6-8-14-11(13)9(3)15-10(...
1,CCCCC(=O)O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ac0b0>,"InChI=1S/C5H10O2/c1-2-3-4-5(6)7/h2-4H2,1H3,(H,..."
2,CC(C)CCC(=O)O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ad0e0>,"InChI=1S/C6H12O2/c1-5(2)3-4-6(7)8/h5H,3-4H2,1-..."
3,C1C2CC(C1C=C2)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73acdd0>,InChI=1S/C8H10O2/c9-8(10)7-4-5-1-2-6(7)3-5/h1-...
4,CCCC(C)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73acd60>,"InChI=1S/C6H12O2/c1-3-4-5(2)6(7)8/h5H,3-4H2,1-..."
...,...,...,...
66,C1=CC=C(C=C1)CC=O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ae110>,"InChI=1S/C8H8O/c9-7-6-8-4-2-1-3-5-8/h1-5,7H,6H2"
67,CC\C=C/CC=O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ae180>,"InChI=1S/C6H10O/c1-2-3-4-5-6-7/h3-4,6H,2,5H2,1..."
68,CCCC(=O)OCC,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ae1f0>,"InChI=1S/C6H12O2/c1-3-5-6(7)8-4-2/h3-5H2,1-2H3"
69,CCC1=NC=C(N=C1C)C,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ae260>,"InChI=1S/C8H12N2/c1-4-8-7(3)10-6(2)5-9-8/h5H,4..."


In [6]:
def embed(mol):
    mol_with_H = Chem.AddHs(mol)
    Chem.AllChem.EmbedMolecule(mol_with_H)
    Chem.AllChem.MMFFOptimizeMolecule(mol_with_H)
    return mol_with_H

mols['rdmol_optimized'] = mols.rdmol.map(embed)

mols

,SMILES,rdmol,inchi,rdmol_optimized
0,CCCCOC(=O)C(C)OC(=O)CCC,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ac200>,InChI=1S/C11H20O4/c1-4-6-8-14-11(13)9(3)15-10(...,<rdkit.Chem.rdchem.Mol object at 0x7f95d73af760>
1,CCCCC(=O)O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ac0b0>,"InChI=1S/C5H10O2/c1-2-3-4-5(6)7/h2-4H2,1H3,(H,...",<rdkit.Chem.rdchem.Mol object at 0x7f95d73afc30>
2,CC(C)CCC(=O)O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ad0e0>,"InChI=1S/C6H12O2/c1-5(2)3-4-6(7)8/h5H,3-4H2,1-...",<rdkit.Chem.rdchem.Mol object at 0x7f95d73aef80>
3,C1C2CC(C1C=C2)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73acdd0>,InChI=1S/C8H10O2/c9-8(10)7-4-5-1-2-6(7)3-5/h1-...,<rdkit.Chem.rdchem.Mol object at 0x7f95d73af840>
4,CCCC(C)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73acd60>,"InChI=1S/C6H12O2/c1-3-4-5(2)6(7)8/h5H,3-4H2,1-...",<rdkit.Chem.rdchem.Mol object at 0x7f95d73af6f0>
...,...,...,...,...
66,C1=CC=C(C=C1)CC=O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ae110>,"InChI=1S/C8H8O/c9-7-6-8-4-2-1-3-5-8/h1-5,7H,6H2",<rdkit.Chem.rdchem.Mol object at 0x7f95d73d5850>
67,CC\C=C/CC=O,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ae180>,"InChI=1S/C6H10O/c1-2-3-4-5-6-7/h3-4,6H,2,5H2,1...",<rdkit.Chem.rdchem.Mol object at 0x7f95d73d58c0>
68,CCCC(=O)OCC,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ae1f0>,"InChI=1S/C6H12O2/c1-3-5-6(7)8-4-2/h3-5H2,1-2H3",<rdkit.Chem.rdchem.Mol object at 0x7f95d73d5930>
69,CCC1=NC=C(N=C1C)C,<rdkit.Chem.rdchem.Mol object at 0x7f95d73ae260>,"InChI=1S/C8H12N2/c1-4-8-7(3)10-6(2)5-9-8/h5H,4...",<rdkit.Chem.rdchem.Mol object at 0x7f95d73d59a0>


In [7]:
calc = Calculator(descriptors) # create calculator for all mordred descriptors (can also specify subtype)

df=calc.pandas(mols['rdmol_optimized'])
print(df)

100%|██████████| 67/67 [00:02<00:00, 26.34it/s]

                                                  ABC  \
0   module 'numpy' has no attribute 'float'.\n`np....   
1   module 'numpy' has no attribute 'float'.\n`np....   
2   module 'numpy' has no attribute 'float'.\n`np....   
3   module 'numpy' has no attribute 'float'.\n`np....   
4   module 'numpy' has no attribute 'float'.\n`np....   
..                                                ...   
66  module 'numpy' has no attribute 'float'.\n`np....   
67  module 'numpy' has no attribute 'float'.\n`np....   
68  module 'numpy' has no attribute 'float'.\n`np....   
69  module 'numpy' has no attribute 'float'.\n`np....   
70  module 'numpy' has no attribute 'float'.\n`np....   

                                                ABCGG  nAcid  nBase  \
0   module 'numpy' has no attribute 'float'.\n`np....      0      0   
1   module 'numpy' has no attribute 'float'.\n`np....      1      0   
2   module 'numpy' has no attribute 'float'.\n`np....      1      0   
3   module 'numpy' has no attri

In [8]:
#Replace mordred errors with NaNs then drop those descriptors/columns
df=df.applymap(lambda x: np.nan if type(x) in [mordred.error.Missing,mordred.error.Error] else x)
df=df.dropna(axis=1)
print(df.shape)

(67, 1227)


In [9]:
#Dropping columns that have a single value
non_zero_std = df.std() != 0
df = df [non_zero_std[non_zero_std].index]
print(df.shape)

(67, 984)


In [10]:
#Dropping highly correlated descriptors: If a pair of descriptors have a Pearson correlation coefficient (r) value greater than the threshold then one descriptor will be removed.
threshold=0.95
df_corr = df.corr().abs()
upper = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
df = df.drop(to_drop, axis=1)
print(df.shape)

(67, 409)


In [11]:
#saving the csv list
to_save=pd.concat([mols[['SMILES']], df], axis=1)
to_save

,SMILES,nAcid,nBase,SpAbs_A,SpMax_A,SpMAD_A,LogEE_A,VR1_A,nAromAtom,nAtom,...,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGT10,TopoShapeIndex,TSRW10,mZagreb1
0,CCCCOC(=O)C(C)OC(=O)CCC,0,0,17.250180,2.180397,1.150012,3.536894,86.757752,0,35,...,0.037333,0.025641,0.014842,0.007812,0.009877,0.005,0.474966,0.833333,44.789525,7.083333
1,CCCCC(=O)O,1,0,7.727407,1.931852,1.103915,2.752227,16.819499,0,17,...,0.053333,0.000000,0.000000,0.000000,0.000000,0.000,0.439907,0.666667,31.336140,3.861111
2,CC(C)CCC(=O)O,1,0,8.472136,2.000000,1.059017,2.899228,19.357315,0,20,...,0.080000,0.000000,0.000000,0.000000,0.000000,0.000,0.570476,0.666667,33.811160,4.722222
3,C1C2CC(C1C=C2)C(=O)O,1,0,12.828338,2.435498,1.282834,3.273564,37.401258,0,20,...,0.063016,0.062500,0.000000,0.000000,0.000000,0.000,0.484733,0.666667,56.287001,3.444444
4,CCCC(C)C(=O)O,1,0,8.647053,2.074313,1.080882,2.900310,21.298450,0,20,...,0.040000,0.000000,0.000000,0.000000,0.000000,0.000,0.518175,0.666667,34.323564,4.722222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,C1=CC=C(C=C1)CC=O,0,0,11.384646,2.148961,1.264961,3.080020,30.671730,6,17,...,0.030222,0.030093,0.040000,0.000000,0.000000,0.000,0.279308,1.000000,36.722228,2.861111
67,CC\C=C/CC=O,0,0,8.054679,1.847759,1.150668,2.739193,16.799928,0,17,...,0.026667,0.027778,0.000000,0.000000,0.000000,0.000,0.213472,1.000000,30.257210,3.250000
68,CCCC(=O)OCC,0,0,8.828427,2.000000,1.103553,2.888532,21.904303,0,20,...,0.016000,0.027778,0.000000,0.000000,0.000000,0.000,0.415802,1.000000,33.420942,4.111111
69,CCC1=NC=C(N=C1C)C,0,0,11.945822,2.267184,1.194582,3.197666,32.871605,6,22,...,0.064762,0.059028,0.000000,0.000000,0.000000,0.000,0.533645,1.000000,39.310842,4.333333


In [12]:
to_save.to_csv('MPNN_AGONIST_SMILES.csv', index = True)

## SECOND FILE DESCRIPTOR FOR MPNN

In [17]:
!pip install mordred
!pip install umap

In [13]:
import os,re,sys,pickle,datetime,time,random,itertools
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from scipy import stats
import math
from tqdm import tqdm
import seaborn as sns
import umap
from multiprocessing import freeze_support
import mordred
from mordred import Calculator, descriptors
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA,NMF
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures
import rdkit
from rdkit import Chem
from rdkit.Chem import DataStructs, AllChem, MolFromSmiles, PandasTools, Descriptors, Draw, PropertyMol, rdmolfiles, rdFMCS
from rdkit import RDConfig
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Draw import MolsToGridImage, IPythonConsole, rdMolDraw2D
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions, Font
DrawingOptions.bondLineWidth=1.8
DrawingOptions.includeAtomNumbers=False
size = (150, 150)
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image
randomstate = 42

In [14]:
filepath2 = "/home/ritesh/Desktop/MOLECULES/morgan/data.csv"
mols = pd.read_csv(filepath2)
mols

,SMILES,ODOR
0,CC1(C2CCC(C2)(C1O)C)C,"balsam,floral,honey,sweet,camphor"
1,CCOC(C)OCC,"cream,earth,ether,ethereal,floral,fruit,green,..."
2,CC=O,"aldehydic,coffee,ether,ethereal,fruit,pungent,..."
3,CCCOC(C)OCCc1ccccc1,"ethereal,green,herb,leaf"
4,COc1ccc(cc1)C(C)=O,"anisic,butter,caramel,fruit,nut,sweet,vanilla"
...,...,...
3086,CCC(O)COC1CCCCC1C(C)(C)C,"amber,wood"
3087,CCCCC(O)C(C)CCC,"herb,spice"
3088,CCCCCC#C\C=C\C=C,"fruit,galbanum,green,leaf,violet"
3089,CC(CC(C)=O)SCc1occc1,meat


In [15]:
mols = mols.drop_duplicates(subset=['SMILES'])
mols

,SMILES,ODOR
0,CC1(C2CCC(C2)(C1O)C)C,"balsam,floral,honey,sweet,camphor"
1,CCOC(C)OCC,"cream,earth,ether,ethereal,floral,fruit,green,..."
2,CC=O,"aldehydic,coffee,ether,ethereal,fruit,pungent,..."
3,CCCOC(C)OCCc1ccccc1,"ethereal,green,herb,leaf"
4,COc1ccc(cc1)C(C)=O,"anisic,butter,caramel,fruit,nut,sweet,vanilla"
...,...,...
3086,CCC(O)COC1CCCCC1C(C)(C)C,"amber,wood"
3087,CCCCC(O)C(C)CCC,"herb,spice"
3088,CCCCCC#C\C=C\C=C,"fruit,galbanum,green,leaf,violet"
3089,CC(CC(C)=O)SCc1occc1,meat


In [24]:
import pandas as pd
from rdkit import Chem

mols['SMILES'].fillna('', inplace=True)

# Create the 'rdmol' column by applying the Chem.MolFromSmiles function
mols['rdmol'] = mols['SMILES'].map(lambda x: Chem.MolFromSmiles(x))




In [25]:

mols['inchi'] = mols['rdmol'].map(Chem.MolToInchi)
mols = mols.drop_duplicates(subset="inchi")

print('Number of SMILES after duplicates removed:',len(mols))

mols

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Proton(s) added/removed

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefin

Number of SMILES after duplicates removed: 2933


[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefined stereo

[20:09:46] WARNING: Omitted undefi

,SMILES,ODOR,rdmol,inchi,Is_Valid_SMILES,rdmol_optimized
0,CC1(C2CCC(C2)(C1O)C)C,"balsam,floral,honey,sweet,camphor",<rdkit.Chem.rdchem.Mol object at 0x7f96ae8903c0>,"InChI=1S/C10H18O/c1-9(2)7-4-5-10(3,6-7)8(9)11/...",True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a282e0>
1,CCOC(C)OCC,"cream,earth,ether,ethereal,floral,fruit,green,...",<rdkit.Chem.rdchem.Mol object at 0x7f96ae8905f0>,"InChI=1S/C6H14O2/c1-4-7-6(3)8-5-2/h6H,4-5H2,1-3H3",True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a28270>
2,CC=O,"aldehydic,coffee,ether,ethereal,fruit,pungent,...",<rdkit.Chem.rdchem.Mol object at 0x7f96ae890350>,"InChI=1S/C2H4O/c1-2-3/h2H,1H3",True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a28350>
3,CCCOC(C)OCCc1ccccc1,"ethereal,green,herb,leaf",<rdkit.Chem.rdchem.Mol object at 0x7f96ae8902e0>,InChI=1S/C13H20O2/c1-3-10-14-12(2)15-11-9-13-7...,True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a283c0>
4,COc1ccc(cc1)C(C)=O,"anisic,butter,caramel,fruit,nut,sweet,vanilla",<rdkit.Chem.rdchem.Mol object at 0x7f96ae890270>,InChI=1S/C9H10O2/c1-7(10)8-3-5-9(11-2)6-4-8/h3...,True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a28430>
...,...,...,...,...,...,...
3086,CCC(O)COC1CCCCC1C(C)(C)C,"amber,wood",<rdkit.Chem.rdchem.Mol object at 0x7f96ae283370>,InChI=1S/C14H28O2/c1-5-11(15)10-16-13-9-7-6-8-...,True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a70270>
3087,CCCCC(O)C(C)CCC,"herb,spice",<rdkit.Chem.rdchem.Mol object at 0x7f96ae2833e0>,InChI=1S/C10H22O/c1-4-6-8-10(11)9(3)7-5-2/h9-1...,True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a702e0>
3088,CCCCCC#C\C=C\C=C,"fruit,galbanum,green,leaf,violet",<rdkit.Chem.rdchem.Mol object at 0x7f96ae283450>,"InChI=1S/C11H16/c1-3-5-7-9-11-10-8-6-4-2/h3,5,...",True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a70350>
3089,CC(CC(C)=O)SCc1occc1,meat,<rdkit.Chem.rdchem.Mol object at 0x7f96ae2834c0>,InChI=1S/C10H14O2S/c1-8(11)6-9(2)13-7-10-4-3-5...,True,<rdkit.Chem.rdchem.Mol object at 0x7f96d0a703c0>


In [29]:
def embed(mol):
    if mol.GetNumAtoms() == 0:
        print("Error: Empty molecule encountered.")
        return None

    mol_with_H = Chem.AddHs(mol)
    Chem.AllChem.EmbedMolecule(mol_with_H)
    Chem.AllChem.MMFFOptimizeMolecule(mol_with_H)
    return mol_with_H

# Apply the embed() function to the mols DataFrame
mols['rdmol_optimized'] = mols.rdmol.map(embed)

# Check for None values (i.e., failed optimization)
empty_molecules = mols[mols['rdmol_optimized'].isnull()]


In [30]:
calc = Calculator(descriptors) # create calculator for all mordred descriptors (can also specify subtype)
df = calc.pandas(mols['rdmol_optimized'].dropna()) # Drop any rows with None values
print(df)


100%|██████████| 2932/2932 [01:42<00:00, 28.58it/s] 


                                                    ABC  \
0     module 'numpy' has no attribute 'float'.\n`np....   
1     module 'numpy' has no attribute 'float'.\n`np....   
2     module 'numpy' has no attribute 'float'.\n`np....   
3     module 'numpy' has no attribute 'float'.\n`np....   
4     module 'numpy' has no attribute 'float'.\n`np....   
...                                                 ...   
3086  module 'numpy' has no attribute 'float'.\n`np....   
3087  module 'numpy' has no attribute 'float'.\n`np....   
3088  module 'numpy' has no attribute 'float'.\n`np....   
3089  module 'numpy' has no attribute 'float'.\n`np....   
3090  module 'numpy' has no attribute 'float'.\n`np....   

                                                  ABCGG  nAcid  nBase  \
0     module 'numpy' has no attribute 'float'.\n`np....      0      0   
1     module 'numpy' has no attribute 'float'.\n`np....      0      0   
2     module 'numpy' has no attribute 'float'.\n`np....      0      0   

In [31]:
#Replace mordred errors with NaNs then drop those descriptors/columns
df=df.applymap(lambda x: np.nan if type(x) in [mordred.error.Missing,mordred.error.Error] else x)
df=df.dropna(axis=1)
print(df.shape)

(2932, 1018)


In [32]:
#Dropping columns that have a single value
non_zero_std = df.std() != 0
df = df [non_zero_std[non_zero_std].index]
print(df.shape)

(2932, 888)


In [33]:
#Dropping highly correlated descriptors: If a pair of descriptors have a Pearson correlation coefficient (r) value greater than the threshold then one descriptor will be removed.
threshold=0.95
df_corr = df.corr().abs()
upper = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
df = df.drop(to_drop, axis=1)
print(df.shape)

(2932, 398)


In [34]:
#saving the csv list
to_save=pd.concat([mols[['SMILES']], df], axis=1)
to_save

,SMILES,nAcid,nBase,SpAbs_A,SpMax_A,SpMAD_A,LogEE_A,VE1_A,VR1_A,nAromAtom,...,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,SRW05,TSRW10,WPath
0,CC1(C2CCC(C2)(C1O)C)C,0.0,0.0,13.412165,2.595011,1.219288,3.388635,3.100165,38.528333,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.744635,3.044522,59.809921,125.0
1,CCOC(C)OCC,0.0,0.0,8.828427,2.000000,1.103553,2.888532,2.598076,21.904303,0.0,...,0.027778,0.000000,0.000000,0.000000,0.000000,0.00000,0.415802,0.000000,33.420942,75.0
2,CC=O,0.0,0.0,2.828427,1.414214,0.942809,1.849457,1.707107,3.363586,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.250000,0.000000,17.310771,4.0
3,CCCOC(C)OCCc1ccccc1,0.0,0.0,18.906808,2.167548,1.260454,3.573727,3.369615,105.265221,6.0,...,0.018308,0.014204,0.007330,0.007595,0.005125,0.00687,0.293479,0.000000,45.146515,462.0
4,COc1ccc(cc1)C(C)=O,0.0,0.0,13.619695,2.250875,1.238154,3.286693,3.095000,39.103451,6.0,...,0.074653,0.027211,0.000000,0.000000,0.000000,0.00000,0.515614,0.000000,40.567492,162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3086,CCC(O)COC1CCCCC1C(C)(C)C,0.0,0.0,19.153277,2.365359,1.197080,3.658748,3.392946,112.976878,0.0,...,0.037037,0.027279,0.014994,0.005102,0.000000,0.00000,0.564090,0.000000,47.933172,455.0
3087,CCCCC(O)C(C)CCC,0.0,0.0,12.824422,2.133003,1.165857,3.220019,2.913900,43.460607,0.0,...,0.015873,0.010204,0.015625,0.000000,0.000000,0.00000,0.455288,0.000000,38.967611,182.0
3088,CCCCCC#C\C=C\C=C,0.0,0.0,13.191508,1.931852,1.199228,3.202455,3.100954,38.458677,0.0,...,0.009259,0.008163,0.007812,0.008230,0.010000,0.00000,0.145210,0.000000,37.236738,220.0
3089,CC(CC(C)=O)SCc1occc1,0.0,0.0,15.514990,2.195484,1.193461,3.446016,3.261773,59.472827,5.0,...,0.015802,0.024093,0.014031,0.015625,0.000000,0.00000,0.443351,2.397895,55.767435,289.0


In [35]:
to_save.to_csv('MPNN_DATA_SMILES.csv', index = True)

In [38]:
import pandas as pd
from rdkit import Chem
import networkx as nx

# Load CSV files
dataset1_path = "/home/ritesh/Desktop/MOLECULES/morgan/MPNN_AGONIST_SMILES.csv"
dataset2_path = "/home/ritesh/Desktop/MOLECULES/morgan/MPNN_DATA_SMILES.csv"

dataset1_df = pd.read_csv(dataset1_path)
dataset2_df = pd.read_csv(dataset2_path)

# Function to convert SMILES to RDKit Mol object
def smiles_to_mol(smiles):
    if isinstance(smiles, str):
        return Chem.MolFromSmiles(smiles)
    else:
        return None

# Function to extract descriptors as a list from DataFrame row
def extract_descriptors(row):
    return row.drop('SMILES').values.tolist()


In [39]:

# Function to construct the graph for each dataset
def construct_graphs(df):
    graphs = []
    for i, row in df.iterrows():
        smiles = row['SMILES']
        descriptors = extract_descriptors(row)
        mol = smiles_to_mol(smiles)
        if mol is None:
            print(f"Skipping row {i} with invalid SMILES: {smiles}")
            continue
        
        G = nx.Graph()

        # Add nodes to the graph for each atom with descriptors as node features
        for atom in mol.GetAtoms():
            atom_idx = atom.GetIdx()
            atom_type = atom.GetAtomicNum()
            G.add_node(atom_idx, atom_type=atom_type, descriptors=descriptors)

        # Add edges to the graph for each bond
        for bond in mol.GetBonds():
            atom1_idx = bond.GetBeginAtomIdx()
            atom2_idx = bond.GetEndAtomIdx()
            G.add_edge(atom1_idx, atom2_idx, bond_type=bond.GetBondTypeAsDouble())

        graphs.append(G)
    return graphs

# Construct graphs for dataset1 and dataset2
graphs_dataset1 = construct_graphs(dataset1_df)
graphs_dataset2 = construct_graphs(dataset2_df)


Skipping row 959 with invalid SMILES: nan
